<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/09preprocesado_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
### Rafa Caballero

##  Preprocesado

Veamos cómo se pueden utilizar las características de Pandas para preprocesado básico de los datos



### Índice
[Duplicados](#Duplicados)<br>
[Ver nulos](#Ver-nulos)<br>
[Eliminar nulos](#Eliminar-nulos)<br>
[Tranformación de valores](#Tranformación-de-valores)<br>
[Normalización y estandarización](#normalizacion)<br>


<a name="Duplicados"></a>
### Duplicados

A menudo nos interesaría eliminar datos duplicados ya sean filas o columnas que pueden influir negativamente en nuestros resultados.

In [ ]:
import pandas as pd

data = pd.DataFrame({'k1': ['a'] * 3 + ['b'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

El método `duplicated` devuelve una serie de booleanos que indican si la fila correspondiente está duplicada

In [ ]:
data.duplicated()

Vemos que tiene toda la apariencia de un filtro; ahora es fácil eliminar duplicados usando el operador not para arrays ~

In [ ]:
data[~data.duplicated()]

Otra forma de hacerlo

In [ ]:
data.drop_duplicates()

Se puede limitar el borrado a ciertas columnas

In [ ]:
data['v1'] = range(7)
data

In [ ]:
data.drop_duplicates(['k1'])

<a name="Ver-nulos"></a>
### Ver nulos

Lo primero que hay que indicar es que en general los valores NaN/missing/nulos no son un problema para Pandas, la mayoría de las funciones los aceptan sin problemas.

Sin embargo, en la librería de machine learning `sklearn` sí puede dar error, por lo que puede que queramos eliminarlos.

Empezamos creando un ejemplo:

In [1]:
import pandas as pd
import numpy as np
datos= [
    [601166, 20111231,  601166,  np.nan,   np.nan, np.nan],
    [600036, 20111231,  600036,  np.nan,    12, np.nan],
    [600016, 20111231,  600016,  4.3,   np.nan, np.nan],
    [601009, 20111231,  601009,  np.nan,   np.nan, np.nan],
    [601939, 20111231,  601939,  2.5,  8],
    [100001, 20111231,  100001,  np.nan,   np.nan, np.nan],
    [np.nan, np.nan,  np.nan,  np.nan,   np.nan, np.nan]

]

df = pd.DataFrame(datos,columns=['A','B','C','D','E','F'])
df

,A,B,C,D,E,F
0,601166.0,20111231.0,601166.0,NaN,NaN,NaN
1,600036.0,20111231.0,600036.0,NaN,12.0,NaN
2,600016.0,20111231.0,600016.0,4.3,NaN,NaN
3,601009.0,20111231.0,601009.0,NaN,NaN,NaN
4,601939.0,20111231.0,601939.0,2.5,8.0,NaN
5,100001.0,20111231.0,100001.0,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN


Lo primero sería contar el número de nulos por columna

In [2]:
df.isnull().sum()

A    1
B    1
C    1
D    5
E    5
F    7
dtype: int64

**Ejercicio 1** ¿Cómo contar el total de nulos?

También podemos verlo por filas con axis; axis=0 se refiere a filas y axis=1 a columnas

In [3]:
df.isna().sum(axis=1) # axis =1 número de columnas nulas en cada fila

0    3
1    2
2    2
3    3
4    1
5    3
6    6
dtype: int64

Filas con algún nulo

In [4]:
(df.isna().sum() > 0).sum()

6

Número de columnas con algún nulo

In [5]:
(df.isna().sum(axis=1) > 0).sum()

7

¿Qué columnas tienen nulos?

In [7]:
df.columns[df.isnull().sum() >0]

Index(['A', 'B', 'C', 'D', 'E', 'F'], dtype='object')

Número de columnas con todo nulos

In [8]:
(df.isna().sum() == df.shape[0]).sum()

1

Qué columnas tienen todo a nulos

In [9]:
df.columns[df.isna().sum() == df.shape[0]]

Index(['F'], dtype='object')


Sobre todo en el caso de dataframes con gran cantidad de datos utilizar una visualización adecuada puede ayudar a entender el origen de los nulos. La biblioteca missingno puede ser muy útil en este sentido

In [ ]:
#!pip install missingno

In [ ]:

import pandas as pd
import missingno as msno
%matplotlib inline

url="https://github.com/RafaelCaballero/tdm/blob/master/datos/tusa2020.csv?raw=true"
df = pd.read_csv(url)
df

In [ ]:
df.info()

Vemos que `coordinates`solo tiene nulos; podemos quitarla directamente

In [ ]:
df2 = df.drop(columns=["coordinates"])

Un "histograma de nulos":

In [ ]:
msno.bar(df)

Otra forma, que puede resultar más informativa

In [ ]:
msno.matrix(df)

En este gráfico los nulos aparecen como líneas blancas ¿llama algo la atención al comparar unas columnas con otras?

El método `heatmap` nos ayudará a relacionar columnas con nulos entre sí:

In [ ]:
msno.heatmap(df)

El valor 1 indica que los nulos suceden a la vez. ne la misma fila, y el -1 que lo hacen en filas contrarias

<a name="Eliminar-nulos"></a>
### Eliminar nulos

Por defecto, `dropna` borra las filas que tienen algún NaN

In [ ]:
df2 = df.dropna()
df2

Problema: hemos borrado el dataframe entero. Una alternativa más conservadora es borrar filas tienen todas sus columnas a  NaN

In [ ]:
df.dropna(how="all")

Ahora no hemos borrado ninguno!! todas las filas tienen algún componente no nulo

**Ejercicio 2** También se pueden borrar columnas añadiendo axis=1 para indicar que trabaje por columnas. Hacerlo:

También se pueden conservar solo las que tienen a partir un cierto número de valores distintos de 0

In [ ]:
df.dropna(thresh=7) # probar a poner 6

Finalmente, podemos indicar que borre solo las que tiene nulos en una cierta columna

In [ ]:
df.dropna(subset=['lang'])

**Comentario**  La eliminación de nulos puede ser complicada; quizás perdiendo información ya que al quitar los nulos quitamos también otros datos potencialmente valiosos. Es conveniente hacer un estudio detallado de dónde aparecen los nulos y qué efectos tienen las distintas medidas que pueden ser



*   Eliminarlos, hay que ver si por filas, por columnas o por ambas y en qué orden
*   Sustituirlo por un valor adecuado, por ejemplo la media de los valores o la interpolación mediante una función. **Nunca** sustituirlos por un valor mágico, que afectará a nuestros análisis




**Ejercicio 3**
Separar el dataframe df2 en dos, uno df_RT con los valores RT_source distinto de  null y otro df_original con  los valores RT_source a null

**Ejercicio 4**

---



¿Eliminarías alguna columna de alguno de los dos ficheros por tener nulos?

**Consejo** En ocasiones es mejor no eliminar a priori los nulos porque puede ser que hagamos distintos análisis con distintas columnas, en ese caso eliminaremos solo tras obtener el dataframe auxiliar conl as columnas necesarias, reduciendo la pérdida de información

<a name="Transformación-de-valores"></a>
### Tranformación de valores

Empezamos con un ejemplo de asignaturas y los créditos que representan

In [ ]:
data = {'nombre': ['A', 'B', 'C', 'D','E', 'F', 'G'],
        'creditos': [4,  6,  4.5, 6,  6, 4.5,  12]}
df = pd.DataFrame(data)
df

**Ejercicio 6** Suponiendo que un crédito son 10 horas lectivas, añadir una columna que 'horas' con los créditos multiplicados por 10

Un caso especial: reemplazar todos los valores concretos. Supongamos que todas las asignaturas de 12 créditos pasan a ser de 16. Podemos hacer

In [ ]:
df2 = df.replace(12, 16)
df2['horas'] = df2.creditos*10
df2

Supongamos ahora que queremos añadir una columna 'tipo' que vale 'optativa' para todas las asignaturas de 4.5 créditos o menos y 'troncal' para asignaturas de más de 4.5 créditos.

Lo primero será escribir una función que haga esta transformación:

In [ ]:
def calcula_tipo(creditos):
    if creditos>4.5:
        r = 'troncal'
    else:
        r = 'optativa'
    return r

Ahora la aplicamos a columna créditos con la función `map` que aplica una función a todos los elementos de una columna:

In [ ]:
df['tipo'] = df.creditos.map(calcula_tipo)
df

También se puede hacer directamente con una función lambda:

In [ ]:
df['tipo2'] = df.creditos.map(lambda x:'optativa' if x<=4.5 else 'troncal')
df

<a name="normalizacion"></a>
### Normalización y estandarización

Vamos a ver estas dos formas de ajustar los datos con un ejemplo


In [ ]:
import pandas as pd

fich = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/madrid/contmettraf.csv"
df = pd.read_csv(fich,parse_dates=["FECHAH"])
df.index = df["FECHAH"]
df

#### Normalización

Supongamos que queremos hacer una gráfica sencilla de NO y CO

Primero vamos a escribir código para obtener un dataframe `dfEne21` que solo tenga las filas de df que correspondan al mes 1 de 2021. Esto se puede hacer filtrando el mes y el año (se utiliza .dt. para indcar a pandas que es una fecha)

In [ ]:
filtro = (df.FECHAH.dt.month == 1) & (df.FECHAH.dt.year == 2021)
dfEne21 = df[filtro]
dfEne21


**Ejercicio 8** Escribir código para obtener un dataframe df2 a partir de `dfEne21` que solo tenga la columna "CO" y "NO". Eliminar  además de df2 todas las filas con algún nulo

Podemos hacer la gráfica así de fácil

In [ ]:
df2.iloc[100:300,:].plot()

¿Cuál es el problema? que está en escalas diferentes. y el CO apenas se ve. Vamos a *normalizar* las columnas con la fórmula

X_new = (X - X_min)/(X_max - X_min)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

metodo = MinMaxScaler()
escalador = metodo.fit(df2)
escalados = escalador.transform(df2)
df3 = pd.DataFrame(escalados,columns=df2.columns)
df3

In [ ]:
df3.describe()

In [ ]:
df3.iloc[100:300,:].plot()

### Estandarización

Para algunos tests y comparaciones nos puede interesar que las distribuciones tengan media 0 y desviación típica 0. Esto se hace con la fórmula
X_new = (X - mean)/Std

In [ ]:
df[ ["TEMPERATURA","HUMEDAD RELATIVA"]].hist(bins=40)

In [ ]:
from sklearn.preprocessing import StandardScaler
df2 = df[["TEMPERATURA","HUMEDAD RELATIVA"]].dropna()
metodo = StandardScaler()
escalador = metodo.fit(df2)
escalados = escalador.transform(df2)
df3 = pd.DataFrame(escalados,columns=df2.columns)
df3.hist(bins=40)
